In [ ]:

# Required Imports
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import streamlit as st

# Load movie metadata
movies = pd.read_csv("movie.csv")
genome_scores = pd.read_csv("genome_scores.csv")
ratings = pd.read_csv("rating.csv")

# --- Tag-Based Recommendation ---
tag_matrix = genome_scores.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)
tag_matrix['movieId'] = tag_matrix.index
tag_matrix = tag_matrix.reset_index(drop=True)
tag_matrix = pd.merge(tag_matrix, movies[['movieId', 'title']], on='movieId')
tag_matrix.set_index('title', inplace=True)
tag_matrix.drop(columns='movieId', inplace=True)
similarity_df = pd.DataFrame(cosine_similarity(tag_matrix), index=tag_matrix.index, columns=tag_matrix.index)

def recommend(movie_title, top_n=5):
    if movie_title not in similarity_df.index:
        return ["Movie not found."]
    return list(similarity_df[movie_title].sort_values(ascending=False)[1:top_n+1].index)

# --- Collaborative Filtering Recommendation ---
user_movie_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)
collab_similarity_df = pd.DataFrame(
    cosine_similarity(user_movie_matrix),
    index=user_movie_matrix.index,
    columns=user_movie_matrix.index
)
id_to_title = dict(zip(movies.movieId, movies.title))
title_to_id = {v: k for k, v in id_to_title.items()}

def recommend_collab(movie_title, top_n=5):
    if movie_title not in title_to_id:
        return ["Movie not found."]
    movie_id = title_to_id[movie_title]
    sim_scores = collab_similarity_df.loc[movie_id].sort_values(ascending=False)[1:top_n+1]
    return [id_to_title[mid] for mid in sim_scores.index]

# --- Genre-Based Recommendation ---
movies['genres'] = movies['genres'].str.replace('|', ' ').fillna('')
genre_matrix = TfidfVectorizer().fit_transform(movies['genres'])
genre_similarity_df = pd.DataFrame(
    cosine_similarity(genre_matrix),
    index=movies['title'],
    columns=movies['title']
)

def recommend_genre(movie_title, top_n=5):
    if movie_title not in genre_similarity_df:
        return ["Movie not found."]
    return list(genre_similarity_df[movie_title].sort_values(ascending=False)[1:top_n+1].index)

# --- Streamlit App ---
st.title("🎬 Rashmi Recommends")
movie_list = sorted(movies['title'].dropna().unique())
selected_movie = st.selectbox("Choose a movie:", movie_list)

if st.button("Show Recommendations"):
    st.subheader("🔖 Tag-Based Recommendations")
    for title in recommend(selected_movie):
        st.markdown(f"• {title}")

    st.subheader("👥 Collaborative Recommendations")
    for title in recommend_collab(selected_movie):
        st.markdown(f"• {title}")

    st.subheader("🎭 Genre-Based Recommendations")
    for title in recommend_genre(selected_movie):
        st.markdown(f"• {title}")
